In [2]:
from getdata import GetDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
data,target = GetDataset(CreateDataFrame=True).Outputdata()

In [58]:
x = np.linspace(data[0][0].min(),data[0][0].max(),900)
data2 = np.zeros((data.shape[0],900))
for i in range(data.shape[0]):
    data2[i,:] = np.interp(x,data[i][0][1:],data[i][1][1:])



In [93]:
np.sqrt(901)
dim = int(np.sqrt(data2.shape[1]))
DataC = np.zeros((data2.shape[0],2,dim,dim))

for i in range(DataC.shape[0]):
    DataC[i,0] = data2[i].reshape((1,dim,dim))
    DataC[i,1] = np.gradient(data2[i]).reshape((1,dim,dim))